In [11]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import nltk
import math
import string
from nltk.corpus import stopwords
from collections import Counter
from nltk.stem.porter import *
from sklearn.feature_extraction.text import TfidfVectorizer
import jieba

In [12]:
df=pd.read_csv("整合資料(年).csv")
df1=df.drop("Unnamed: 0", axis=1)
df1

,index,日期,週期ID,斷詞結果,月份,詞彙統計
0,0,2021-06-09,2021-06-2,"['○', '九年', '四月', '由金', '管會', '催生', '基金', '平台'...",2021-06,"Counter({'基金': 10, '實驗': 8, '投資': 6, '這場': 5, ..."
1,1,2021-06-09,2021-06-2,"['界面', '新聞', '報導', '閆衍', '日', '參加', '2021', '年...",2021-06,"Counter({'違約': 11, '新增': 7, '企業': 6, '年': 5, '..."
2,2,2021-06-03,2021-06-1,"['加入', '強', '基金', '前', '存股', '一族', '理財', '投資',...",2021-06,"Counter({'基金': 20, '配息': 17, '再': 8, '元': 7, '..."
3,3,2021-06-03,2021-06-1,"['基金', '月報', '揭露', '前', '十大', '持股', '掌握', '了解'...",2021-06,"Counter({'基金': 6, '請問': 6, '全': 5, 'B': 5, 'C'..."
4,4,2021-06-02,2021-06-1,"['請', '問版', '手邊', '一些', '南非幣', '目前', '匯率', '不錯...",2021-06,"Counter({'南非幣': 9, '換回': 7, '台幣': 7, '請': 4, '..."
...,...,...,...,...,...,...
2314,2315,2019-09-04,2019-09-2,"['退休', '專案', '理念', '比一比', '退休', '專案', '產品', '核...",2019-09,"Counter({'退休': 20, '基金': 15, '好': 13, '日子': 11..."
2315,2316,2019-08-14,2019-08-3,"['安聯', '收益', '成長', '基金', '投資', '投資', '策略', '有別...",2019-08,"Counter({'基金': 24, '投資': 10, '這檔': 10, '報酬': 8..."
2316,2317,2019-07-04,2019-07-1,"['貝萊德', '亞洲', '老虎債', '裡面', '買', '同類', '基金', '績...",2019-07,"Counter({'亞洲': 22, '債券': 21, '基金': 15, '貝萊德': ..."
2317,2318,2018-03-06,2018-03-2,"['配息', '率什麼', '含息', '報酬率', '波動度', '夏普', '值新手',...",2018-03,"Counter({'基金': 31, '配息': 20, '報酬率': 17, '報酬': ..."


In [13]:
def tool(str):
    return(eval(str))

In [14]:
df1["斷詞結果"]=df1["斷詞結果"].apply(tool)

In [15]:
df1

,index,日期,週期ID,斷詞結果,月份,詞彙統計
0,0,2021-06-09,2021-06-2,"[○, 九年, 四月, 由金, 管會, 催生, 基金, 平台, 基富通, 執行, 好享退, ...",2021-06,"Counter({'基金': 10, '實驗': 8, '投資': 6, '這場': 5, ..."
1,1,2021-06-09,2021-06-2,"[界面, 新聞, 報導, 閆衍, 日, 參加, 2021, 年, 中信, 用風險, 展望, ...",2021-06,"Counter({'違約': 11, '新增': 7, '企業': 6, '年': 5, '..."
2,2,2021-06-03,2021-06-1,"[加入, 強, 基金, 前, 存股, 一族, 理財, 投資, 太, 懂, 存到, 錢, 買,...",2021-06,"Counter({'基金': 20, '配息': 17, '再': 8, '元': 7, '..."
3,3,2021-06-03,2021-06-1,"[基金, 月報, 揭露, 前, 十大, 持股, 掌握, 了解, 基金, 組成, 重要, 資料...",2021-06,"Counter({'基金': 6, '請問': 6, '全': 5, 'B': 5, 'C'..."
4,4,2021-06-02,2021-06-1,"[請, 問版, 手邊, 一些, 南非幣, 目前, 匯率, 不錯, 請, 問是, 直接, 換回...",2021-06,"Counter({'南非幣': 9, '換回': 7, '台幣': 7, '請': 4, '..."
...,...,...,...,...,...,...
2314,2315,2019-09-04,2019-09-2,"[退休, 專案, 理念, 比一比, 退休, 專案, 產品, 核心, 比一比, 基金, 選擇,...",2019-09,"Counter({'退休': 20, '基金': 15, '好': 13, '日子': 11..."
2315,2316,2019-08-14,2019-08-3,"[安聯, 收益, 成長, 基金, 投資, 投資, 策略, 有別於, 同類, 基金, 配息, ...",2019-08,"Counter({'基金': 24, '投資': 10, '這檔': 10, '報酬': 8..."
2316,2317,2019-07-04,2019-07-1,"[貝萊德, 亞洲, 老虎債, 裡面, 買, 同類, 基金, 績效, 一比, 費用, 率, 比...",2019-07,"Counter({'亞洲': 22, '債券': 21, '基金': 15, '貝萊德': ..."
2317,2318,2018-03-06,2018-03-2,"[配息, 率什麼, 含息, 報酬率, 波動度, 夏普, 值新手, 買, 基金, 大可不必, ...",2018-03,"Counter({'基金': 31, '配息': 20, '報酬率': 17, '報酬': ..."


In [16]:
dict_text=df1["斷詞結果"].tolist()

In [18]:
pip install gensim

Note: you may need to restart the kernel to use updated packages.


In [19]:
import gensim
from gensim import corpora
data = dict_text
#透過gensim以text_data建立字典
dictionary =corpora.Dictionary(data)

corpus = [dictionary.doc2bow(text) for text in data]

In [20]:
NUM_TOPICS  = 7
ldamodel = gensim.models.ldamodel.LdaModel(corpus=corpus, num_topics = NUM_TOPICS ,id2word=dictionary,passes=40 ,chunksize=150)
# ldamodel.save('Topic7.gensim')

In [21]:
topics = ldamodel.print_topics(num_words=6)
for topic in topics:
    print(topic)

(0, '0.038*"基金" + 0.015*"買" + 0.011*"配息" + 0.009*"投資" + 0.008*"好" + 0.006*"月"')
(1, '0.011*"退休" + 0.011*"人" + 0.009*"理財" + 0.009*"錢" + 0.007*"投資" + 0.006*"做"')
(2, '0.011*"房價" + 0.007*"房屋" + 0.006*"房市" + 0.005*"商圈" + 0.005*"貸款" + 0.004*"台中"')
(3, '0.020*"市場" + 0.016*"美國" + 0.012*"債券" + 0.009*"經濟" + 0.009*"疫情" + 0.008*"全球"')
(4, '0.039*"投資" + 0.023*"基金" + 0.013*"公司" + 0.012*"投資人" + 0.009*"年" + 0.008*"企業"')
(5, '0.011*"台灣" + 0.008*"銀行" + 0.008*"年" + 0.007*"月" + 0.007*"金融" + 0.006*"元"')
(6, '0.002*"吳欣儒" + 0.002*"股本" + 0.002*"台新金" + 0.002*"吳東進" + 0.002*"保德信" + 0.001*"國票金"')


In [23]:
pip install pyLDAvis

  Using cached pyLDAvis-3.3.1-py2.py3-none-any.whl
  Created wheel for future: filename=future-0.18.2-py3-none-any.whl size=491070 sha256=c5f65673160b9a29540748760293b3d8b6fbe8eae7c207c14f0af5b31b46acc2
  Stored in directory: c:\users\88696\appdata\local\pip\cache\wheels\8e\70\28\3d6ccd6e315f65f245da085482a2e1c7d14b90b30f239e2cf4
Successfully built future
Note: you may need to restart the kernel to use updated packages.


In [24]:
import pyLDAvis.gensim_models
import pyLDAvis
pyLDAvis.enable_notebook()

LDAvis_prepared = pyLDAvis.gensim_models.prepare(ldamodel, corpus, dictionary,mds='mmds')

C:\Users\88696\.conda\envs\myenv\lib\site-packages\pyLDAvis\_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(


In [25]:
LDAvis_prepared

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0      0.026298 -0.276780       1        1  30.993923
3     -0.220495 -0.218057       2        1  28.537421
5     -0.282901  0.053390       3        1  15.079552
4     -0.145692  0.133064       4        1  14.192218
1      0.231476 -0.141859       5        1  10.356759
2      0.231377  0.219270       6        1   0.705971
6      0.159936  0.230971       7        1   0.134156, topic_info=     Term          Freq         Total Category  logprob  loglift
50     基金  15531.000000  15531.000000  Default  30.0000  30.0000
78     投資  10384.000000  10384.000000  Default  29.0000  29.0000
1051   美國   4714.000000   4714.000000  Default  28.0000  28.0000
479     買   4740.000000   4740.000000  Default  27.0000  27.0000
208    市場   6811.000000   6811.000000  Default  26.0000  26.0000
...   ...           ...           ...      ...      ...      ...
725    億股      0.474199      1.244140   Topic7  -7.8867   5.6493
750    出價      0.468979      1.240019   Topic7  -7.8978   5.6416
1042  經營權      0.466573      1.235609   Topic7  -7.9029   5.6400
764    前董      0.418535      1.188715   Topic7  -8.0116   5.5701
716   保德信      2.188036     15.174025   Topic7  -6.3576   4.6773

[462 rows x 6 columns], token_table=       Topic      Freq  Term
term                        
9431       2  0.152760    \n
9431       4  0.844199    \n
9432       4  0.981728  \r\n
14913      7  0.570615   105
4859       1  0.168615  2019
...      ...       ...   ...
1264       5  0.037010     高
2083       1  0.105310  高收益債
2083       2  0.894143  高收益債
12176      6  0.921537    高雄
6527       4  0.989836    黃金

[797 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 4, 6, 5, 2, 3, 7])

In [23]:
pyLDAvis.save_html(LDAvis_prepared, 'newlda_01.html')

C:\Users\88696\.conda\envs\py38\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [26]:
import pickle
ldamodel.save('allyear_01.gensim')
dictionary.save('allyeardict_01.gensim')
pickle.dump(corpus,open('allyear_01.pkl','wb'))

In [27]:
def find_topic(word):
    dict2={}
    
    new_doc = word
    new_doc_bow = dictionary.doc2bow(new_doc)
    dict1=ldamodel.get_document_topics(new_doc_bow)
    dict2={}
    for i in dict1:
        dict2[str(i[0])]=i[1]
    return max(dict2,key=dict2.get)

In [30]:
pip install matplotlib

  Using cached cycler-0.11.0-py3-none-any.whl (6.4 kB)
  Using cached kiwisolver-1.4.2-cp38-cp38-win_amd64.whl (55 kB)
  Using cached Pillow-9.1.0-cp38-cp38-win_amd64.whl (3.3 MB)
Note: you may need to restart the kernel to use updated packages.
